In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

url = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv"
data = pd.read_csv(url)
data = data.dropna()

X = data['median_income'].values.reshape(-1, 1)
y = data['median_house_value'].values.reshape(-1, 1)


taking the input as it is 

In [10]:
m = len(X)
split = int(0.8 * m)

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


This is known as train test split.
Train-test split is crucial in machine learning to prevent overfitting, providing an unbiased evaluation of a model's real-world performance by testing it on unseen data, ensuring it learns general patterns rather than memorizing noise, and helping select the best model for new situations. It splits data into training (for learning) and testing (for evaluation) sets, mimicking deployment and assessing true predictive power. 

In [11]:
def polynomial_features(X, degree):
    X_poly = X
    for d in range(2, degree + 1):
        X_poly = np.c_[X_poly, X ** d]
    return X_poly
degree = 3
X_train_poly = polynomial_features(X_train, degree)
X_test_poly = polynomial_features(X_test, degree)

making a polynomial feature and testing it and below it is feature scaling and adding a bias term.

In [12]:
mean = X_train_poly.mean(axis=0)
std = X_train_poly.std(axis=0)

X_train_poly = (X_train_poly - mean) / std
X_test_poly = (X_test_poly - mean) / std

X_train_poly = np.c_[np.ones((len(X_train_poly), 1)), X_train_poly]
X_test_poly = np.c_[np.ones((len(X_test_poly), 1)), X_test_poly]


In [13]:
def train_polynomial_regression(X, y, lr=0.01, epochs=3000, lambda_=0.1):
    m, n = X.shape
    theta = np.zeros((n, 1))

    for _ in range(epochs):
        y_pred = X @ theta
        error = y_pred - y

        gradients = (2/m) * X.T @ error
        gradients[1:] += 2 * lambda_ * theta[1:]  # bias term is already added in the first term and second term does not contain the bias term

        theta -= lr * gradients

    return theta

Gradient descent with l2 regularization after that we would require to tarin the model as polynomials may require testing after training.

In [14]:
theta = train_polynomial_regression(
    X_train_poly, y_train,
    lr=0.01,
    epochs=3000,
    lambda_=0.1
)


In [15]:
def rmse(X, y, theta):
    return np.sqrt(np.mean((X @ theta - y) ** 2))

train_rmse = rmse(X_train_poly, y_train, theta)
test_rmse = rmse(X_test_poly, y_test, theta)

print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)


Train RMSE: 84276.96953270337
Test RMSE: 83599.99492906051


RMSE evaluation and after that extra plots to visualize the polynomial curves